In [1]:
from transformers import BertModel, BertTokenizer, AdamWeightDecay, get_linear_schedule_with_warmup
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from textwrap import wrap

c:\Users\jhoan\Documents\BERT_Model\BERT_NLP\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Inicialization of the BERT model
RANDOM_SEED = 42
MAX_LEN = 200
BATCH_SIZE = 16
DATASET_PATH = "historias_clinicas_procesadas.xlsx"
NCLASS = 6

np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda:0


In [3]:
import torch
torch.cuda.is_available()

True

In [4]:
import torch
print(torch.cuda.is_available())
print(torch.version.cuda)

True
11.8


In [5]:
# Cargar dataset

df = pd.read_excel(DATASET_PATH)
df.head()


,sexo,edad,grupo,especialidad_medica,subjetivo,objetivo,concatenada,sexo_codificado,grupo_codificado
0,Femenino,38,Otros trastornos,PSICOLOGÍA,"Paciente refiere: ""Me empezaron a dar como uno...","Paciente alerta, colaboradora con apariencia o...","['empezar', 'episodio', 'tom', 'pastilla', 'do...",0,0
1,Masculino,22,T. externalizantes,PSICOLOGÍA,"Paciente refiere ""Me he sentido muy mal, en el...","Paciente a quien evaluó por primera vez, alert...","['sentido', 'trabajar', 'concentrar yo', 'cosa...",1,4
2,Masculino,9,Otros trastornos,PSICOLOGÍA,"La madre refiere ""el viene por un acompañamien...",NaN,"['madre', 'venir', 'acompañamiento', 'emociona...",1,0
3,Masculino,28,Otros trastornos,PSICOLOGÍA,"Paciente refiere ""Estas cosas que han pasado m...","Paciente quien evalúo por primera vez, alerta,...","['cosa', 'problema', 'empresa', 'ocasión', 'sa...",1,0
4,Femenino,8,Otros trastornos,PSICOLOGÍA,"La madre refiere ""Ella ha manifestado ciertas ...","Paciente ingresa en compañía de la madre, aler...","['madre', 'manifestar', 'conducta', 'palabrase...",0,0


In [6]:
# Probamos con 500 registros utilizando sample y Ramdom Seed
df = df.sample(n=500, random_state=RANDOM_SEED).reset_index(drop=True)

In [7]:
df.head()

,sexo,edad,grupo,especialidad_medica,subjetivo,objetivo,concatenada,sexo_codificado,grupo_codificado
0,Masculino,16,T. de adaptación,PSICOLOGÍA,SEGUIMIENTO: La paciente refiere altibajos emo...,"Paciente orientada en tiempo, lugar y persona,...","['seguimiento', 'altibajo', 'emocional', 'pone...",1,1
1,Femenino,36,T. de ansiedad,PSICOLOGÍA,"""mi mamá estuvo conmigo dos semanas, la verdad...","Paciente alerta, elocuente, colaboradora, con ...","['mam', 'semana', 'cosa', 'matrimonio', 'segui...",0,2
2,Femenino,37,T. depresivos,PSICOLOGÍA,"Paciente refiere: \n""La semana pasada estuvo d...",Se realiza consulta de seguimiento por psicolo...,"['semana', 'duro', 'pasar', 'cosa', 'man', 'ch...",0,3
3,Femenino,18,T. externalizantes,PSICOLOGÍA,"Cita #2- 17/06/25\n\n""Bien, ya en vacaciones, ...","Paciente alerta, orientada en las tres esferas...","['cita', 'vacación', 'logr', 'salvar', 'materi...",0,4
4,Femenino,65,Otros trastornos,PSICOLOGÍA,"me he sentido muy triste, muy mal estos dias p...","Paciente quien evalúo por primera vez, con bas...","['sentido', 'triste', 'dar', 'esposo', 'person...",0,0


In [8]:
# Calcular la longitud máxima de las historias clínicas en términos de número de palabras
max_lenght = df["concatenada"].apply(lambda x: len(x.split())).max()

print(f"La longitud máxima de las historias clínicas es: {max_lenght} palabras.")

La longitud máxima de las historias clínicas es: 330 palabras.


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   sexo                 500 non-null    object
 1   edad                 500 non-null    int64 
 2   grupo                500 non-null    object
 3   especialidad_medica  500 non-null    object
 4   subjetivo            497 non-null    object
 5   objetivo             497 non-null    object
 6   concatenada          500 non-null    object
 7   sexo_codificado      500 non-null    int64 
 8   grupo_codificado     500 non-null    int64 
dtypes: int64(3), object(6)
memory usage: 35.3+ KB


In [10]:
df.head(10)

,sexo,edad,grupo,especialidad_medica,subjetivo,objetivo,concatenada,sexo_codificado,grupo_codificado
0,Masculino,16,T. de adaptación,PSICOLOGÍA,SEGUIMIENTO: La paciente refiere altibajos emo...,"Paciente orientada en tiempo, lugar y persona,...","['seguimiento', 'altibajo', 'emocional', 'pone...",1,1
1,Femenino,36,T. de ansiedad,PSICOLOGÍA,"""mi mamá estuvo conmigo dos semanas, la verdad...","Paciente alerta, elocuente, colaboradora, con ...","['mam', 'semana', 'cosa', 'matrimonio', 'segui...",0,2
2,Femenino,37,T. depresivos,PSICOLOGÍA,"Paciente refiere: \n""La semana pasada estuvo d...",Se realiza consulta de seguimiento por psicolo...,"['semana', 'duro', 'pasar', 'cosa', 'man', 'ch...",0,3
3,Femenino,18,T. externalizantes,PSICOLOGÍA,"Cita #2- 17/06/25\n\n""Bien, ya en vacaciones, ...","Paciente alerta, orientada en las tres esferas...","['cita', 'vacación', 'logr', 'salvar', 'materi...",0,4
4,Femenino,65,Otros trastornos,PSICOLOGÍA,"me he sentido muy triste, muy mal estos dias p...","Paciente quien evalúo por primera vez, con bas...","['sentido', 'triste', 'dar', 'esposo', 'person...",0,0
5,Femenino,34,T. de ansiedad,PSICOLOGÍA,"Paciente refiere: ""He estado bien, aunque a ve...","Paciente alerta, colaboradora con apariencia o...","['sentir', 'depresin', 'poner', 'recordar', 'm...",0,2
6,Masculino,22,Otros trastornos,PSICOLOGÍA,"SEGUIMIENTO: El paciente expresa ""bien, traté ...","Paciente quien evalúo por teleconsulta, se evi...","['seguimiento', 'expresar', 'trat', 'seguir', ...",1,0
7,Femenino,23,T. de ansiedad,PSICOLOGÍA,"Paciente refiere:\n""Tuve cita con psiquiatría ...","Paciente alerta y consciente, orientada en per...","['refieretuve', 'cita', 'psiquiatra', 'mand', ...",0,2
8,Femenino,69,T. de ansiedad,PSICOLOGÍA,"14/08/25- Cuarta cita\n\n""Creo que un poquito ...","Paciente alerta, orientado en las tres esferas...","['cuarto', 'citacreo', 'poquito', 'quedar', 'e...",0,2
9,Femenino,86,T. depresivos,PSICOLOGÍA,La acompañante refiere: “mi abuela viene hace ...,"Paciente orientada en tiempo, lugar y persona,...","['acompañante', 'abuela', 'venir', 'dar', 'dol...",0,3


In [11]:
# Removemos los corchetes
df["concatenada"] = df["concatenada"].apply(lambda x: str(x).replace("[","").replace("]",""))

# Removemos las comillas simples
df["concatenada"] = df["concatenada"].apply(lambda x: str(x).replace("'",""))

# Unimos las palabras separadas por comas
df["concatenada"] = df["concatenada"].apply(lambda x: str(x).replace(","," "))

In [12]:
df.head()

,sexo,edad,grupo,especialidad_medica,subjetivo,objetivo,concatenada,sexo_codificado,grupo_codificado
0,Masculino,16,T. de adaptación,PSICOLOGÍA,SEGUIMIENTO: La paciente refiere altibajos emo...,"Paciente orientada en tiempo, lugar y persona,...",seguimiento altibajo emocional poner escri...,1,1
1,Femenino,36,T. de ansiedad,PSICOLOGÍA,"""mi mamá estuvo conmigo dos semanas, la verdad...","Paciente alerta, elocuente, colaboradora, con ...",mam semana cosa matrimonio seguir sentir ...,0,2
2,Femenino,37,T. depresivos,PSICOLOGÍA,"Paciente refiere: \n""La semana pasada estuvo d...",Se realiza consulta de seguimiento por psicolo...,semana duro pasar cosa man charlar poda ...,0,3
3,Femenino,18,T. externalizantes,PSICOLOGÍA,"Cita #2- 17/06/25\n\n""Bien, ya en vacaciones, ...","Paciente alerta, orientada en las tres esferas...",cita vacación logr salvar materia tenar ...,0,4
4,Femenino,65,Otros trastornos,PSICOLOGÍA,"me he sentido muy triste, muy mal estos dias p...","Paciente quien evalúo por primera vez, con bas...",sentido triste dar esposo persona tiempo ...,0,0


In [13]:
from transformers import BertTokenizer, BertModel

# TOKENIZACION

PRE_TRAINED_MODEL_NAME = "bert-base-multilingual-cased"
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [14]:
# Ejemplos de tokenización

sample_text = "Hola soy un texto de ejemplo para la tokenización."
tokens = tokenizer.tokenize(sample_text)
tokens_ids = tokenizer.convert_tokens_to_ids(tokens)
print(f"Texto original: {sample_text}")
print(f"Tokens: {tokens}")
print(f"IDs de tokens: {tokens_ids}")

Texto original: Hola soy un texto de ejemplo para la tokenización.
Tokens: ['Ho', '##la', 'soy', 'un', 'texto', 'de', 'ejemplo', 'para', 'la', 'tok', '##eni', '##zación', '.']
IDs de tokens: [20220, 10330, 103559, 10119, 27888, 10104, 20223, 10220, 10109, 18436, 18687, 23700, 119]


In [15]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

2.7.1+cu118
True


In [16]:
# Instala PyTorch si no está instalado

# Codificacion para introducir a BERT

encoding = tokenizer.encode_plus(
    sample_text,
    add_special_tokens=True,
    max_length=MAX_LEN,
    return_token_type_ids=False,
    padding="max_length",
    truncation=True,
    return_attention_mask=True,
    return_tensors="pt",
)

In [17]:
encoding.keys()

KeysView({'input_ids': tensor([[   101,  20220,  10330, 103559,  10119,  27888,  10104,  20223,  10220,
          10109,  18436,  18687,  23700,    119,    102,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
     

In [18]:
print(tokenizer.convert_ids_to_tokens(encoding["input_ids"][0]))
print(encoding["input_ids"][0])
print(encoding["attention_mask"][0])

['[CLS]', 'Ho', '##la', 'soy', 'un', 'texto', 'de', 'ejemplo', 'para', 'la', 'tok', '##eni', '##zación', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]',

In [19]:
# CREACION DEL DATASET

class CustomDataset(Dataset):
    def __init__(self, text, labels, tokenizer, max_len):
        self.text = text
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
      return len(self.text)

    def __getitem__(self, item):
      text = str(self.text[item])
      label = self.labels[item]
      encoding = tokenizer.encode_plus(
          text,
          add_special_tokens=True,
          max_length=self.max_len,
          return_token_type_ids=False,
          pad_to_max_length=True,
          truncation=True,
          return_attention_mask=True,
          return_tensors="pt",
      )
      return {"text": text,
                "input_ids": encoding["input_ids"].flatten(),
                "attention_mask": encoding["attention_mask"].flatten(),
                "labels": torch.tensor(label, dtype=torch.long)}


In [20]:
df.columns

Index(['sexo', 'edad', 'grupo', 'especialidad_medica', 'subjetivo', 'objetivo',
       'concatenada', 'sexo_codificado', 'grupo_codificado'],
      dtype='object')

In [21]:
# Data Loader:

def data_loader(df, tokenizer, max_len, batch_size):
    dataset = CustomDataset(
        text = df["concatenada"].to_numpy(),
        labels = df["grupo_codificado"].to_numpy(),
        tokenizer = tokenizer,
        max_len = MAX_LEN
    )
    return DataLoader(dataset, batch_size = BATCH_SIZE, num_workers = 4)


In [22]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=RANDOM_SEED)

train_data_loader = data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE)

In [28]:
# Red Neuronal para el modelo BERT

class BERTTextClassifier(nn.Module):
    def __init__(self, n_classes):
        super(BERTTextClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
        self.dropout = nn.Dropout(0.3)
        self.linear = nn.Linear(self.bert.config.hidden_size, n_classes)

    def forward(self, input_ids, attention_mask):
      outputs = self.bert(
          input_ids = input_ids,
          attention_mask = attention_mask
      )
      pooled_out = outputs.pooler_output
      drop_out = self.dropout(pooled_out)
      output = self.linear(drop_out)
      return output


In [29]:
model = BERTTextClassifier(NCLASS)
model = model.to(device)

In [30]:
print(model)

BERTTextClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, eleme

In [31]:
# ENTRENAMIENTO

EPOCHS = 5
optimizer = optim.AdamW(model.parameters(), lr=2e-5)
# total_steps = (train_data_loader.batch_size) * EPOCHS
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps = 0,
    num_training_steps = total_steps
)

loss_fn = nn.CrossEntropyLoss().to(device)

In [32]:
# Iteracion de Entranamiento

def train_model(model, data_loader, loss_fn, optimizer, device, scheduler, n_examples):
    model = model.train()
    losses = []
    correct_predictions = 0
    for batch in data_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(input_ids = input_ids, attention_mask = attention_mask)
        _, preds = torch.max(outputs, dim=1)
        loss = loss_fn(outputs, labels)
        correct_predictions += torch.sum(preds == labels)
        losses.append(loss.item())
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
    return correct_predictions.double() / n_examples, np.mean(losses)

def eval_model(model, data_loader, loss_fn, device, n_examples):
    model = model.eval()
    losses = []
    correct_predictions = 0
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids = input_ids, attention_mask = attention_mask)
            _, preds = torch.max(outputs, dim=1)
            loss = loss_fn(outputs, labels)
            correct_predictions += torch.sum(preds == labels)
            losses.append(loss.item())
    return correct_predictions.double() / n_examples, np.mean(losses)

In [33]:
len(df_train)

400

In [ ]:
import time

# Entrenamiento del Modelo

for epoch in range(EPOCHS):
  print("Epoch {} de {}".format(epoch + 1, EPOCHS))
  print("-----------------------------------")
  
  start_time = time.time()

  train_acc, train_loss = train_model(model, train_data_loader, loss_fn, optimizer, device, scheduler, len(df_train))

  test_acc, test_loss = eval_model(model, test_data_loader, loss_fn, device, len(df_test))
  
  end_time = time.time()

  print("Entramiento: Loss: {}, accuracy: {}".format(train_loss, train_acc))
  print("Evaluacion: Loss: {}, accuracy: {}".format(test_loss, test_acc))
  print(f"Tiempo de la época: {end_time - start_time:.2f} segundos")
  print("")


Epoch 1 de 5
-----------------------------------
